# Performance Fixes

## 2025.4.21

Found that the cluster priority implementation is not stable with different weights.
The current implementation uses the following equation for calculating priority:

$$\mathcal{P}(C) = \sum_{S \in \mathcal{B}_C} y_S - \sum_{e \in E_C} \left( \sum_{S \in \mathcal{B}_C | e \in \delta(S)} y_S \right) - |\mathcal{O}_B| - |E_C| $$

We need to change a few things:
1. change $ - |\mathcal{O}_B| - |E_C|$ to $/ (|\mathcal{O}_B| + |E_C|)^3$ because the cardinality doesn't have the same unit
2. change $e \in E_C$ to $e \in \mathcal{E}_C$, the local parity factor solution.
3. change $\sum_{S \in \mathcal{B}_C | e \in \delta(S)} y_S$ to $w_e$ because that's how the primal-dual gap defined. Although this is perhaps not very important since the parity factor only uses tight edges $\mathcal{E}_C \subseteq T$ and tight edges, by definition, has a dual sum equal to $w_e$.

The new priority calculation is below:

$$\mathcal{P}(C) = \left( \sum_{S \in \mathcal{B}_C} y_S - \sum_{e \in \mathcal{E}_C} w_e \right) / \left( |\mathcal{O}_B| - |E_C| \right)^3 $$

The performance comparison data is below:

```sh
python3 -m qec_lego_bench generate-samples 'rsc(d=5,p=0.001)' './dist/rsc-d5-p001' --shots 500000 --decoder 'mwpf'
```

```sh
for i in {1..3}
do
  echo "Running command $i:"
  time python3 -m qec_lego_bench benchmark-samples ./dist/rsc-d5-p001 --decoder 'mwpf'
done
Running command 1:
decoding time: 3.148e-04s, elapsed: 1.574e+02s, shots: 500000
logical error rate: 44/500000 = 8.800e-05
python3 -m qec_lego_bench benchmark-samples ./dist/rsc-d5-p001 --decoder   149.62s user 12.07s system 100% cpu 2:40.22 total
Running command 2:
decoding time: 3.169e-04s, elapsed: 1.584e+02s, shots: 500000
logical error rate: 44/500000 = 8.800e-05
python3 -m qec_lego_bench benchmark-samples ./dist/rsc-d5-p001 --decoder   148.95s user 12.47s system 100% cpu 2:40.37 total
Running command 3:
decoding time: 3.126e-04s, elapsed: 1.563e+02s, shots: 500000
logical error rate: 44/500000 = 8.800e-05
python3 -m qec_lego_bench benchmark-samples ./dist/rsc-d5-p001 --decoder   147.57s user 12.21s system 100% cpu 2:38.29 total
```


After the fix:

```sh
for i in {1..3}
do
  echo "Running command $i:"
  time python3 -m qec_lego_bench benchmark-samples ./dist/rsc-d5-p001 --decoder 'mwpf'
done
Running command 1:
decoding time: 2.837e-04s, elapsed: 1.419e+02s, shots: 500000
logical error rate: 44/500000 = 8.800e-05
python3 -m qec_lego_bench benchmark-samples ./dist/rsc-d5-p001 --decoder   136.56s user 10.41s system 101% cpu 2:25.01 total
Running command 2:
decoding time: 2.924e-04s, elapsed: 1.462e+02s, shots: 500000
logical error rate: 43/500000 = 8.600e-05
python3 -m qec_lego_bench benchmark-samples ./dist/rsc-d5-p001 --decoder   141.72s user 9.00s system 101% cpu 2:27.78 total
Running command 3:
decoding time: 2.803e-04s, elapsed: 1.401e+02s, shots: 500000
logical error rate: 44/500000 = 8.800e-05
python3 -m qec_lego_bench benchmark-samples ./dist/rsc-d5-p001 --decoder   134.74s user 9.87s system 102% cpu 2:21.74 total
```